# This is used to generate twiddle factors for Kyber.

In [1]:
import math
import random
def EX_GCD(a,b,arr): #扩展欧几里得
    if b == 0:
        arr[0] = 1
        arr[1] = 0
        return a
    g = EX_GCD(b, a % b, arr)
    t = arr[0]
    arr[0] = arr[1]
    arr[1] = t - int(a / b) * arr[1]
    return g
def ModReverse(a,n): #ax=1(mod n) 求a模n的乘法逆x
    arr = [0,1,]
    gcd = EX_GCD(a,n,arr)
    if gcd == 1:
        return (arr[0] % n + n) % n
    else:
        return -1

# Constants for Modulus 769

In [2]:
M=769
M2=3329
n=16
N=32
r=2**n
R=2**N
alpha=5
alpha2=2**alpha
Mprime_mont=ModReverse(-M,r)
Mprime_plant=ModReverse(M,R)
Mprime_modM2=ModReverse(M,M2)
Mont_const=r%M
Plant_const=(-R)%M
Q1pr=11
omegaQ1=Q1pr**((M-1)//256)%M
invNQ1=ModReverse(128,M)

print("q",M)
print("qa",M*alpha2)
print("omegaQ1, invNQ1", omegaQ1, invNQ1)
Mont_const_inv=ModReverse(Mont_const,M)
Plant_const_inv=ModReverse(Plant_const,M)
print("Mont_const_inv",Mont_const_inv)
print("Plant_const=-2^2n mod M",Plant_const)
print("Mont_const=2^n mod M",Mont_const)
print("Mprime_modM2",Mprime_modM2)
print("Mprime_plant=M^-1 mod 2^2n",Mprime_plant)
print("Plant_const*Mprime_plant= q^-1 (-2^2n) mod 2^2n",Plant_const*Mprime_plant%R)

q 769
qa 24608
omegaQ1, invNQ1 562 763
Mont_const_inv 9
Plant_const=-2^2n mod M 750
Mont_const=2^n mod M 171
Mprime_modM2 1000
Mprime_plant=M^-1 mod 2^2n 3842571521
Plant_const*Mprime_plant= q^-1 (-2^2n) mod 2^2n 5585134


# Basic modular arithmetic for computing NTT twiddle factors

In [14]:

# Plantard reduction
def plant_red(a):
	t=(((((a*Mprime_plant)%R)//r)+alpha2)*M//r)
	return t
def plant_mul(a,b):
	return plant_red(a*b)
def plant_mul_ntt(a,zeta):
	t=(((((a*zeta)%R)//r)+alpha2)*M//r)
	return t

In [16]:
def init_ntt_c_plant_from_mont(zetas,type):
	plant_ntt_asm=[]
	plant_ntt_asm_prime=[]
	Mont_inv=ModReverse(Mont_const,M)
	for zeta in zetas:
		t=Plant_const*zeta*Mont_inv%M
		plant_ntt_asm.append(t)
	if type==1: 
		# for inverse NTT. Finalize the final two twiddle factors in the final layer as (Plant^2*128^-1)%M 
		plant_ntt_asm[-2]=plant_ntt_asm[-2]*Plant_const*Mont_inv%M
		plant_ntt_asm[-1]=plant_ntt_asm[-1]*Plant_const*Mont_inv%M
	for zeta in plant_ntt_asm: 
		# zeta*Mprime_plant %R
		t=(zeta*Mprime_plant)%R
		plant_ntt_asm_prime.append(t)
	if type==0:
		print("plant_ntt_asm",plant_ntt_asm_prime)
	else:
		print("plant_inv_ntt_asm",plant_ntt_asm_prime)

zetas=[-23, 112, -151, -134, -52, -148, 227, 232,
-71, 212, 236, 21, 341, 379, -202, -220,
352, 292, 238, 145, 194, -276, 70, -274,
117, 333, 66, 247, -237, -83, -252, -244,
331, -241, 167, 357, -355, 291, -358, 105, -115, -209, 14, 99, -260, 29, 366, -378, -318, 278, 353, 354, -184, 127, 330, -303, 222, -78, -348, -44, 201, 158, 350, 168]
init_ntt_c_plant_from_mont(zetas,0)

zetas_asm=[-164, -81, 361, 186, -3, -250, -120, -308, 129, -16, -223, -362, -143, 131, -337,
-76, 147, -114, -23, 112, -151, -134,
-98, -272, 54, -52, -148, 227, 232,
36, -2, -124, -71, 212, 236, 21,
-75, -80, -346, 341, 379, -202, -220,
-339, 86, -51, 352, 292, 238, 145,
-255, 364, 267, 194, -276, 70, -274,
282, 161, -15, 117, 333, 66, 247,
-203, 288, 169, -237, -83, -252, -244,
-34, 191, 307, 331, -241, 167, 357,
199, -50, -24, -355, 291, -358, 105,
178, -170, 226, -115, -209, 14, 99,
270, 121, -188, -260, 29, 366, -378,
-10, -380, 279, -318, 278, 353, 354,
149, 180, -375, -184, 127, 330, -303,
369, -157, 263, 222, -78, -348, -44,
-192, -128, -246, 201, 158, 350, 168,0]
init_ntt_c_plant_from_mont(zetas,0)

plant_ntt_asm [3138844760, 1334846793, 999738812, 1854264165, 1681125041, 1150537404, 2820492178, 3071823164, 726067294, 2066499220, 3272887953, 1055590142, 4255871365, 1871019564, 2731130050, 1826338500, 513832239, 1792827701, 3373420347, 2993631302, 1161707670, 3306398751, 3518633806, 3406931146, 1586177780, 3853741788, 3317569017, 3825816122, 971813147, 122872927, 217820188, 619949766, 3753209393, 770748358, 4099487641, 765163225, 3630336467, 1742561504, 3479537875, 982983413, 2809321912, 2379266669, 703726762, 681386230, 4110657907, 1457719720, 1217559000, 2474213930, 1195218468, 1089100940, 564098436, 614364633, 3635921600, 2088839752, 3702943196, 1949211426, 2569161192, 374203913, 3982199847, 2083254619, 1513571050, 3647091866, 413299844, 4149753838]
plant_ntt_asm [3138844760, 1334846793, 999738812, 1854264165, 1681125041, 1150537404, 2820492178, 3071823164, 726067294, 2066499220, 3272887953, 1055590142, 4255871365, 1871019564, 2731130050, 1826338500, 513832239, 1792827701, 33734

In [22]:
def is_prime(num):
    if num==1:
        return False
    else:
        for i in range(2,num):
            if num%i==0:
                return False
    return True

def search_prime():
    n=256
    for i in range(491520,1<<23):
        if is_prime(i)==True and (i-1)%256==0:
            print("find a prime:",i)
search_prime()

find a prime: 493313
find a prime: 494849
find a prime: 495361
find a prime: 495617
find a prime: 496897
find a prime: 497153
find a prime: 498689
find a prime: 499969
find a prime: 503297
find a prime: 505601
find a prime: 506113
find a prime: 507137
find a prime: 509441
find a prime: 514049
find a prime: 514561
find a prime: 518657
find a prime: 520193
find a prime: 522497
find a prime: 523777
find a prime: 524801
find a prime: 525313
find a prime: 529153
find a prime: 530177
find a prime: 531457
find a prime: 532993
find a prime: 533249
find a prime: 534529


KeyboardInterrupt: 